# Calling CPP Fortran Module from Python

The objective of this notebook is to show how to use Python to interact with the CPP module (in Fortran). This could serve as a basis for doing something similar with other models. 


We use f2py to pre-compile our fortran cpp module (available at https://github.com/CEDIA-models/cpp). We can then use the module (all its functions and global variables) interactively from Python. Here is an example. 

In [1]:
from cpp import cpp

Now load the parameters from cpp

In [ ]:
cpp.loadcpp();

Now let's create an individual age 60 who is considering retiring. We need to input the birth year and age at which benefits are claimed. The year for now is simply birth year plus age but in future will allow for adjustments to benefits. 

In [ ]:
import numpy as np
byear = 1958;
year = 2018;
age = 60;

Now setup his earnings history. The regime is in place since 1966. Let's assume this retiree has contributed the maximum in each year between 1976 and 2018 (1976 is the year he turned 18). The YMPE is the yearly maximum pensionable earnings for CPP. It is indexed every year to reflect growth in wages and prices (inflation). The function setearn allows to obtain the years in which the individual theoretically could have started to contribute (either at age 18 or 1966) and the maximum age at which he could contribute (either his current age or if larger the late retirement age of 70). In future applications, one can fill years to zero when the individual has not worked in a given year. 

In [ ]:
dim = cpp.setearn(byear,age);
earn = cpp.ympe[np.logical_and(cpp.years>=dim[0], cpp.years<=dim[1])];
earn



The average monthly pensionable earnings or AMPE is the basis for computing CPP benefits. Several steps are involved. 

* Compute the unadjusted pensionable earnings (UPE) for each year (capped at YMPE and zero if below base amount). 
* Express UPE as fraction of YMPE for each year. Call this APE.
* In the year claiming occurs, compute the average YMPE for the last 5 years (including current year). Multiply each APE by this average YMPE. 
* Sum those and divide by number of years to get a temporary APE (there is a disability and 2 child reading dropout provision)
* There is a general dropout provision that 15% of months can be dropped (rounded up). Rest of canada has increased this in recent years.  
* Once all dropouts rights have been computed, The years dropped out are those with the lowest APE.  
* There is also an additional provision for those with earnings past age 65: they can increased their dropout 1/1 for each year if their UPE/YMPE for that year is larger than their AMPE. 


We then call the ampe function from cpp to compute his ampe (in Quebec)

In [ ]:
ampe = cpp.ampe(byear,year,age,earn,True);
ampe


If from other province, we can see that AMPE would be slightly higher because higher exclusion: 

In [ ]:
ampe_cpp = cpp.ampe(byear,year,age,earn,False);
[ampe,ampe_cpp, ampe_cpp/ampe]

Now with the ampe, we can compute the benefit he would obtain if he claimed at age 60

In [ ]:
ben = cpp.ben(ampe,age,byear)
ben


The number found at https://www.rrq.gouv.qc.ca/fr/programmes/regime_rentes/rente_retraite/Pages/montant_rr.aspx is 725.87$.  

We can also investigate what happens if he was to claim at a different age, assuming he does not work and therefore does not contribute more: 

In [ ]:
benp = np.zeros((10,1));
ages = [60, 61, 62, 63, 64, 65, 66, 67 , 68, 69];
i = 0
for a in ages:
    benp[i] = cpp.ben(ampe,a,byear);
    i +=1
benp/benp[5]

We express above benefits as a ratio of the benefit at the normal claiming age (65). We get exactly the same numbers as those in the link above (RRQ). 

Finally, we can decide to lower the delayed retirement credit (after age 65) to check what happens. We need to modify the drc (here we change it for all cohorts)

In [ ]:
cpp.drc[:] = cpp.drc[:]*0.5;

In [ ]:
benm = np.zeros((10,1));
i = 0
for a in ages:
    benm[i] = cpp.ben(ampe,a,byear);
    i +=1
benm/benm[5]

We can finally plot those. 

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig, ax = plt.subplots()
ax.plot(ages,benp)
ax.plot(ages,benm)
ax.set(xlabel='claiming age',ylabel='monthly benefit',title='Illustration of Changing the DRC',xticks=ages)
ax.legend(['baseline','lower DRC'])
ax.grid()
plt.show()

We can look at how the benefit formula changed across cohorts. To do that, 

In [ ]:
cpp.loadcpp();
ben = np.zeros((len(ages),3));
i = 0
for b in [1938,1948,1958]:
    dim = cpp.setearn(b,60);
    earn = cpp.ympe[np.logical_and(cpp.years>=dim[0], cpp.years<=dim[1])];
    avgearn = np.mean(earn[-5:]);
    j = 0
    for a in ages:
        ampe = cpp.ampe(b,b+a,a,earn,True);
        ben[j,i] = cpp.ben(ampe,a,b);
        j+=1
    i+=1    

ben    
        

Let's now play a bit with contributions. We will take our worker and run him trough the contributions calculator. 
We have a function called tax which allows to compute worker contributions to CPP. 

In [ ]:
cpp.loadcpp()
selfearn = earn;
selfearn = np.zeros((len(earn),1));
contrib = np.zeros((len(earn),1));
yr = dim[0];
age = 18;
for i in range(0,len(earn)):
    contrib[i] = cpp.tax(yr,earn[i],selfearn[i],age,True,True);
    yr+=1;
    age+=1;
contrib

In [ ]:
[i,yr,age,dim[0]]